In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import missingno as msno # display missing data

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # Alignments
%matplotlib inline

import seaborn as sns # theme & dataset
print(f"Matplotlib Version : {mpl.__version__}")
print(f"Seaborn Version : {sns.__version__}")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Coloumn Description

* **Suburb** - Suburb

* **Address** - Address

* **Rooms** - Number of rooms

* **Price** - Price in Australian dollars

**Method** - 
    
    S - property sold;
    SP - property sold prior;
    PI - property passed in;
    PN - sold prior not disclosed;
    SN - sold not disclosed;
    NB - no bid;
    VB - vendor bid;
    W - withdrawn prior to auction;
    SA - sold after auction;
    SS - sold after auction price not disclosed.
    N/A - price or highest bid not available.

**Type** -

    br - bedroom(s);
    h - house,cottage,villa, semi,terrace;
    u - unit, duplex;
    t - townhouse;
    dev site - development site;
    o res - other residential.

* **SellerG** - Real Estate Agent

* **Date** - Date sold

* **Distance** - Distance from CBD in Kilometres

* **Regionname** - General Region (West, North West, North, North east …etc)

* **Propertycount** - Number of properties that exist in the suburb.

* **Bedroom2** - Scraped # of Bedrooms (from different source)

* **Bathroom** - Number of Bathrooms

* **Car** - Number of carspots

* **Landsize** - Land Size in Metres

* **BuildingArea** - Building Size in Metres

* **YearBuilt** - Year the house was built

* **CouncilArea** - Governing council for the area

* **Lattitude** - Self explanitory

* **Longtitude** - Self explanitory

# Understand Data

* **Number of rows/entries** - 34857 entries, 0 to 34856

* **Data columns** (total 21 columns):

*   **Numerical** (13 columns)
 *    float64 (12 columns) - 
    'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
    'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
    'Propertycount'

 *    int64 (1 column) - 'Rooms'

* **Nominal/Categorical** (8 columns)
 *     object (8 columns) - 'Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea', 'Regionname'
 

### Which features could higly be relative to Price?

Inference - Distance, Landsize, BuildingArea, YearBuilt, Rooms


In [6]:
# show the current working directory
%pwd 

In [7]:
%ls

In [8]:
df = pd.read_csv('/kaggle/input/melbourne-housing-market/Melbourne_housing_FULL.csv')
df.shape # 34857 rows, 21 columns

In [9]:
df.head()

In [10]:
# df.dtypes
df.info()

In [11]:
# Statistical Summary
df.describe()

In [12]:
# Summary of object features
# df.describe(include = 'object')
df.describe(include = [np.object])

In [13]:
# the number of values that each unique value has
df['Type'].value_counts()
df['Price'].value_counts().head(10)

There are not many empty values in **categorical features**, but there are a lot of empty values in **numerical features**.

In [14]:
print(df.isnull().sum())
print('')
print(df.isnull().sum()/len(df))

In [15]:
msno.matrix(df)

# EDA

* Missing values
* Outliers


### Separate the dataset into categorical and numerical.

In [16]:
cat_cols = [col for col in df if df[col].dtype == 'O']

print('Number of categorical values: ',len(cat_cols))

df[cat_cols].head()

In [13]:
for col in df[cat_cols]:
    print(f'{col} has {len(df[col].unique())} distict values.')

In [14]:
cp_cat_cols = ['Type', 'Method', 'CouncilArea', 'Regionname']

In [15]:
for col in df[cp_cat_cols]:
    data = df.copy()
    data.groupby(col)['Price'].median().plot.bar()
    plt.xlabel(col)
    plt.ylabel('Price')
    plt.title(col)
    plt.show()

In [18]:
df[cat_cols].isnull().sum()

In [19]:
# Fill missing values with mode in CouncilArea and Regionname
df['CouncilArea'].fillna(df['CouncilArea'].mode()[0], inplace = True)
df['Regionname'].fillna(df['Regionname'].mode()[0], inplace = True)
df[cat_cols].isnull().sum()

In [20]:
num_cols = [col for col in df if df[col].dtype != 'O']

print('Number of numerical values: ',len(num_cols))

df[num_cols].head()

#### Divide numerical values into two groups: discrete and continuous

In [24]:
disc_cols = [col for col in num_cols if len(df[col].unique())<25]
df[disc_cols].head()

In [25]:
cont_cols = [col for col in num_cols if col not in disc_cols]
df[cont_cols].head()

In [29]:
df[num_cols].isnull().sum()

In [19]:
for col in df[cont_cols]:
    print(col, len(df[col].unique())) 

In [20]:
# discrete features and Price

for col in disc_cols:
    data = df.copy()
    data.groupby(col)['Price'].median().plot.bar()
    plt.xlabel(col)
    plt.ylabel('Price')
    plt.title(col)
    plt.show()

In [21]:
# Continuous features - with histogram

for col in cont_cols:
    data = df[col]
    df[col].hist(bins = 25)
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.title(col)
    plt.show()

In [22]:
# Using logarithmic transformation
for col in cont_cols:
    data = df.copy()
    if 0 in data[col].unique():
        pass
    else:
        data[col] = np.log(abs(data[col])) # To change Lattitude to positive, abs is used
        data['Price'] = np.log(data['Price'])
        
        plt.scatter(data[col], data['Price'])
        plt.xlabel(col)
        plt.ylabel('Price')
        plt.title(col)
        plt.show()

#### Outliers

log transformation is used to remove skewness

Many outliers in cont_cols, so median is required to fill the missing values.

mode is used for disc_cols.

In [43]:
# Continuous features
for col in cont_cols:
    data = df.copy()
    data[col] = np.log1p(abs(df[col]))
    data.boxplot(column=col)
    plt.ylabel(col)
    plt.show()

In [41]:
# Discrete features
for col in disc_cols:
    data = df.copy()
    data[col] = np.log1p(df[col])
    data.boxplot(column=col)
    plt.ylabel(col)
    plt.show()

In [56]:
for col in cont_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace = True)

df[cont_cols].isnull().sum()

In [57]:
for col in disc_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace = True)

df[disc_cols].isnull().sum()

In [58]:
df.isnull().sum()